In [95]:
import os
from pathlib import Path
import json
from dotenv import load_dotenv
from pinecone import Pinecone,ServerlessSpec
from pypdf import PdfReader
from langchain.text_splitter import(
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter
)
import re


In [74]:
BASE_DIR = Path(os.getcwd()).parent.parent
PATH_ENV = os.path.join(BASE_DIR,".env")
DATA_DIRECTORY = os.path.join(BASE_DIR,r"2_data_names_spaces")
TOKENS_DIRECTORY = os.path.join(DATA_DIRECTORY,r"investigations\tokens")
load_dotenv(override=True,dotenv_path=PATH_ENV)


True

In [ ]:
path_token_file = os.path.join(TOKENS_DIRECTORY,"tokens1.json")
tokens_file= open(path_token_file,"r",encoding="utf-8")
tokens_json = json.load(tokens_file)

{'first_tokenization': {'values': ['\'"\\\'Maestría en Ingeniería \\\\\\\\nde Sistemas \\\\\\\\ny Computación\\\\\\\\nDissertation\\\\\\\\nConstructing a language for testing\\\\\\\\nReinforcement Learning programs using\\\\\\\\nNLP techniques\\\\\\\\nLUIS ALEJANDRO MEDINA\\\\\\\\nJuly 19, 2025\\\\\\\\nThis thesis is submitted in partial fulfillment of the requirements\\\\\\\\nfor a degree of Master in Systems and Computing Engineering\\\\\\\\n(MISIS).\\\\\\\\nThesis Committee:\\\\\\\\nProf. Nicolás Cardozo (Promotor) Universidad de los Andes, Colombia\\\\\\\\nProf. Reviewer 1Ivana Dusparic Trinity College Dublin, Ireland\\\\\\\\nProf. Reviewer 2Ruben Manrique Universidad de los Andes, Colombia\\\\\\\\n\\\\\\\\nConstructing a language for testing Reinforcement Learning pro-\\\\\\\\ngrams using NLP techniques\\\\\\\\n© YEAR LUIS ALEJANDRO MEDINA\\\\\\\\nSystems and Computing Engineering Department\\\\\\\\nFLAG lab\\\\\\\\nFaculty of Engineering\\\\\\\\nUniversidad de los Andes\\\\\\\\nB

In [53]:
path_pdf_thesis = os.path.join(DATA_DIRECTORY,"investigations\landing\Constructing a language for testing RL.pdf")
reader_thesis = PdfReader(path_pdf_thesis)

<>:1: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\l'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_5916\2082917411.py:1: SyntaxWarning: invalid escape sequence '\l'
  path_pdf_thesis = os.path.join(DATA_DIRECTORY,"investigations\landing\Constructing a language for testing RL.pdf")


In [ ]:
def tokenizer_1(reader_pdf: PdfReader,token_json:dict,description:str) -> dict:
    pdf_texts = [p.extract_text().strip() for p in reader_pdf.pages]
    character_splitter = RecursiveCharacterTextSplitter(
        separators = ["\n\n","\n",". ", " ", ""],
        chunk_size = 1000,
        chunk_overlap=0
    )

    character_split_texts = character_splitter.split_text(
        "\n\n".join(pdf_texts)
    )

    token_json["first_tokenization"] = { 
        "values" : character_split_texts, 
        "description":description
    }


    return token_json


tokens = tokenizer_1(
    reader_pdf=reader_thesis,
    token_json=tokens_json,
    description="tokens with recursive tokenization, without cleaning or another technique"
)



In [100]:
CONTROL_CHARS = ''.join(map(chr, list(range(0,32)) + [127]))
CONTROL_RE = re.compile(f"[{re.escape(CONTROL_CHARS.replace('\\n','').replace('\\t',''))}]")

def strip_control_chars(text:str) -> str:
    return CONTROL_RE.sub("", text)

tokens['first_tokenization']['values'] = [strip_control_chars(text)  for text in tokens['first_tokenization']['values']]

In [101]:
file = open(path_token_file,'w',encoding="utf-8")
json.dump(tokens,file,indent=4)


In [97]:
tokens['first_tokenization']['values']

['\'"\\\'Maestría en Ingeniería \\\\\\\\nde Sistemas \\\\\\\\ny Computación\\\\\\\\nDissertation\\\\\\\\nConstructing a language for testing\\\\\\\\nReinforcement Learning programs using\\\\\\\\nNLP techniques\\\\\\\\nLUIS ALEJANDRO MEDINA\\\\\\\\nJuly 19, 2025\\\\\\\\nThis thesis is submitted in partial fulfillment of the requirements\\\\\\\\nfor a degree of Master in Systems and Computing Engineering\\\\\\\\n(MISIS).\\\\\\\\nThesis Committee:\\\\\\\\nProf. Nicolás Cardozo (Promotor) Universidad de los Andes, Colombia\\\\\\\\nProf. Reviewer 1Ivana Dusparic Trinity College Dublin, Ireland\\\\\\\\nProf. Reviewer 2Ruben Manrique Universidad de los Andes, Colombia\\\\\\\\n\\\\\\\\nConstructing a language for testing Reinforcement Learning pro-\\\\\\\\ngrams using NLP techniques\\\\\\\\n© YEAR LUIS ALEJANDRO MEDINA\\\\\\\\nSystems and Computing Engineering Department\\\\\\\\nFLAG lab\\\\\\\\nFaculty of Engineering\\\\\\\\nUniversidad de los Andes\\\\\\\\nBogotá, Colombia\\\'"\'',
 '\'"\\\'